In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.chart import BarChart, Reference
from docx import Document
from selenium import webdriver
import base64, tempfile, os
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()
out_dir = filedialog.askdirectory(title="Выберите папку для сохранения файлов")
os.makedirs(out_dir, exist_ok=True)

df = pd.read_csv("третья.csv")
df["Стоимость"] = df["Количество"] * df["Цена"]

wb = Workbook()
ws = wb.active
ws.append(list(df.columns))
for row in df.values.tolist():
    ws.append(row)

chart = BarChart()
chart.title = "Стоимость"
data = Reference(ws, min_col=4, min_row=1, max_row=len(df) + 1)
cats = Reference(ws, min_col=1, min_row=2, max_row=len(df) + 1)
chart.add_data(data, titles_from_data=True)
chart.set_categories(cats)
ws.add_chart(chart, "F2")

excel_path = os.path.join(out_dir, "Отчет.xlsx")
wb.save(excel_path)

doc = Document()
doc.add_heading("Отчет по товарам", 0)

table = doc.add_table(rows=1, cols=len(df.columns))
hdr = table.rows[0].cells
for i, c in enumerate(df.columns):
    hdr[i].text = c

for _, row in df.iterrows():
    cells = table.add_row().cells
    for i, c in enumerate(df.columns):
        cells[i].text = str(row[c])

doc.add_page_break()
doc.add_heading("Итоги", level=1)
doc.add_paragraph(f"Итоговая сумма: {df['Стоимость'].sum()}")
max_row = df.loc[df['Цена'].idxmax()]
doc.add_paragraph(f"Самая дорогая позиция: {max_row['Имя']} — {max_row['Цена']} руб.")

word_path = os.path.join(out_dir, "Отчет.docx")
doc.save(word_path)

html = f"""
<html>
<head>
<meta charset="utf-8">
<style>
body {{ font-family: TimesNewRoman; margin: 20px; }}
table, th, td {{ border: 1px solid black; border-collapse: collapse; padding: 5px; }}
</style>
</head>
<body>
<h1>Отчет по товарам</h1>
{df.to_html(index=False)}
<h2>Итоги</h2>
<p>Итоговая сумма: {df['Стоимость'].sum():.2f}</p>
<p>Самая дорогая позиция: {max_row['Имя']} — {max_row['Цена']} руб.</p>
</body>
</html>
"""

tmp_html = tempfile.NamedTemporaryFile(delete=False, suffix=".html", mode="w", encoding="utf-8")
tmp_html.write(html)
tmp_html.close()

driver = webdriver.Edge()
driver.get("file://" + tmp_html.name)

pdf = driver.execute_cdp_cmd("Page.printToPDF", {"printBackground": True})
pdf_path = os.path.join(out_dir, "Отчет.pdf")
with open(pdf_path, "wb") as f:
    f.write(base64.b64decode(pdf["data"]))
driver.quit()

print("\nГотово!")
print("Excel:", excel_path)
print("Word: ", word_path)
print("PDF:  ", pdf_path)
